In [23]:
# Import Dependencies
import pandas as pd
from sqlalchemy import create_engine
import csv
import json
import numpy as np
import pymongo

In [24]:
# Read CSV into dataframe
brew_csv = "../data/beer_reviews.csv"
brewer_df = pd.read_csv(brew_csv)

craft_csv = "../data/craft_beer_fest_data3.csv"
craft_df = pd.read_csv(craft_csv)

# style_csv = "../../../gorbas/beerStyles.csv"
# style_df = pd.read_csv(style_csv)

# rename columns for merging later
craft_df.rename(columns={'Beer':'beer_name','Brewery':'brewery_name'}, inplace=True)

#drop duplicates from this data set
craft_df = craft_df.drop_duplicates(subset="beer_name", keep='first', inplace=False)

style_df.rename(columns={'type':'Style'}, inplace=True)

In [25]:
group_by_beer_name_mean = brewer_df.groupby('beer_name').mean()
group_by_beer_name_mean = group_by_beer_name_mean.reset_index()
# group_by_beer_name_mean.head(20)

raw_beer_df = pd.merge(group_by_beer_name_mean, craft_df, on=["beer_name"], how='outer')
raw_beer_df = raw_beer_df.dropna(how='any').reset_index(drop=True)
raw_beer_df = raw_beer_df.drop(columns=['brewery_id', 'review_time','Table','Ratings','ABV','URL','beer_beerid','Score','Bro Score'])

# print(al_beer_df.columns.values)

avg_scores = list(raw_beer_df["review_overall"])
avg_overall_scores = list(raw_beer_df["rAvg"])
weighted_averages = []

for av, ov in zip(avg_scores, avg_overall_scores):
    weighted_avg = (av+ov)/2
    weighted_averages.append(weighted_avg)

raw_beer_df['Avg Overall Score'] = weighted_averages
al_beer_df = raw_beer_df.drop(columns=['review_overall', 'rAvg'])

# al_beer_df = pd.merge(raw_beer_df, style_df, on=["Style"], how='outer')
# al_beer_df['group'].value_counts()
# al_beer_df['group'].isna().sum()

al_beer_df.head(20)


,beer_name,review_aroma,review_appearance,review_palate,review_taste,beer_abv,brewery_name,City,State,Style,Avg Overall Score
0,#9,3.434466,3.501040,3.398058,3.396671,5.100000,Magic Hat Brewing Company,South Burlington,VT,Fruit / Vegetable Beer,3.474764
1,Allagash Interlude,4.237569,4.074586,4.145028,4.254144,9.500000,Allagash Brewing Company,Portland,ME,Saison / Farmhouse Ale,4.183895
2,Allagash White,4.020246,4.029049,3.992958,4.128521,5.200000,Allagash Brewing Company,Portland,ME,Witbier,4.200977
3,Amber,3.260577,3.530769,3.373077,3.403846,4.740909,Abita Brewing Co.,Abita Springs,LA,Vienna Lager,3.508942
4,Amber Road Amber Ale,3.525000,3.950000,3.625000,3.550000,5.500000,Baxter Brewing Co.,Lewiston,ME,American Amber / Red Ale,3.642500
5,Apollo,3.807692,3.923077,3.961538,3.961538,5.500000,Sixpoint Brewery,Brooklyn,NY,Kristalweizen,4.014872
6,Baggywrinkle Barleywine,3.600000,3.600000,3.300000,3.428571,12.000000,Cisco Brewers Inc.,Nantucket,MA,English Barleywine,3.387857
7,Be Hoppy,4.150000,4.183333,4.050000,4.116667,6.500000,Wormtown Brewing Co.,Worcester,MA,American IPA,4.218333
8,Belgian White,3.528497,3.512953,3.448187,3.466321,4.725882,Long Trail Brewing Co.,Bridgewaters Corners,VT,Witbier,3.538109
9,Black Hops Beer,3.467391,3.739130,3.576087,3.576087,6.750000,Blue Hills Brewery,Canton,MA,American Black Ale,3.581522


In [26]:
def get_average_scores(column, dfToFilter):
    group_by_current = dfToFilter.groupby(column).mean()

    counts_df = dfToFilter[column].value_counts().to_frame(name=None).reset_index()
    counts_df = counts_df.rename(index=str, columns={"index": column, column: "Quantity"})

    with_counts_df = pd.merge(group_by_current, counts_df, on=[column], how='outer')
    with_counts_df = with_counts_df.rename(index=str, columns={"review_aroma":"Aroma","review_appearance":"Appearance",
                                                                       "review_palate":"Palate","review_taste":"Taste",
                                                                       "beer_abv":"ABV(%)"})
    with_counts_df = with_counts_df.transpose()
    return with_counts_df

def create_mongo(column, groupedDfToFilter):
    final_json = groupedDfToFilter.to_json()
    print(final_json)

    mongo_db = pymongo.MongoClient('localhost', 27017)['Brews']

    db_to_add = mongo_db[column]
    db_to_add.remove()
    db_to_add.insert(json.loads(final_json))

In [27]:
# grouped_df.to_csv("brewery_names_summary.csv", encoding="utf-8", index=False)
columnsToGroup = ["brewery_name","City","State","Style"]
dfsForMongo = []

for column in columnsToGroup:
    final_grouped_df = get_average_scores(column, al_beer_df)
#     print(final_grouped_df.head())
    dfsForMongo.append(final_grouped_df)

dfsForMongo

[                                   0                         1  \
 brewery_name       Abita Brewing Co.  Allagash Brewing Company   
 Aroma                        3.21973                   4.12891   
 Appearance                   3.39113                   4.05182   
 Palate                       3.23819                   4.06899   
 Taste                        3.25432                   4.19133   
 ABV(%)                        4.4703                      7.35   
 Avg Overall Score            3.37702                   4.19244   
 Quantity                           2                         2   
 
                                                  2                      3  \
 brewery_name       Anderson Valley Brewing Company  Avery Brewing Company   
 Aroma                                      3.92331                3.76776   
 Appearance                                 3.99276                 3.8261   
 Palate                                     3.92824                3.71021   
 Tast

In [28]:
for column,df in zip(columnsToGroup, dfsForMongo):
    create_mongo(column,df)

{"0":{"brewery_name":"Abita Brewing Co.","Aroma":3.2197345693,"Appearance":3.3911331184,"Palate":3.2381851681,"Taste":3.2543182865,"ABV(%)":4.4703030303,"Avg Overall Score":3.3770235491,"Quantity":2},"1":{"brewery_name":"Allagash Brewing Company","Aroma":4.1289077698,"Appearance":4.0518174656,"Palate":4.0689926854,"Taste":4.1913323866,"ABV(%)":7.35,"Avg Overall Score":4.1924360702,"Quantity":2},"2":{"brewery_name":"Anderson Valley Brewing Company","Aroma":3.9233104067,"Appearance":3.9927631579,"Palate":3.9282446172,"Taste":4.055173445,"ABV(%)":6.1633333333,"Avg Overall Score":3.9553513756,"Quantity":2},"3":{"brewery_name":"Avery Brewing Company","Aroma":3.7677636322,"Appearance":3.8260957397,"Palate":3.7102143403,"Taste":3.7374497739,"ABV(%)":7.0306060606,"Avg Overall Score":3.7819358374,"Quantity":3},"4":{"brewery_name":"Ballast Point Brewing Company","Aroma":4.1122522927,"Appearance":4.1714374,"Palate":4.085077333,"Taste":4.1640224218,"ABV(%)":8.2,"Avg Overall Score":4.1276730353,"Qu

C:\Users\cflint\Continuum\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:21: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
C:\Users\cflint\Continuum\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:22: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


In [ ]:
for index, row in df.iterrows():
    print(row['c1'], row['c2'])